In [13]:
import os
import json
import pandas as pd


os.chdir('/Users/annamcmanus/Documents/2024-25 Masters Year/Dissertation/scottish_gaelic_chatbot/llm/')
os.getcwd()



'/Users/annamcmanus/Documents/2024-25 Masters Year/Dissertation/scottish_gaelic_chatbot/llm'

In [14]:
# Path to llama_finetuned directory
base_dir = "model_results/llama_finetuned"

timestamps = ["2025-07-10_23-03-02",
              "2025-07-10_22-55-42"]

result_folders = [os.path.join(base_dir, f"results_{ts}") for ts in timestamps if os.path.isdir(os.path.join(base_dir, f"results_{ts}"))]

# Prepare data storage
gaelic_data = {}
english_data = {}

# Initialize once; prompts assumed to be the same across runs
gaelic_prompts = []
english_prompts = []

for folder in sorted(result_folders):
    json_path = os.path.join(folder, "metadata.json")
    if not os.path.isfile(json_path):
        print(f"Warning: No metadata.json in {folder}")
        continue

    with open(json_path, "r", encoding="utf-8") as f:
        metadata = json.load(f)

    # Extract completions
    gaelic = metadata.get("gaelic_completions", [])
    english = metadata.get("english_completions", [])

    if not gaelic_prompts:
        gaelic_prompts = [entry["prompt"] for entry in gaelic]
    if not english_prompts:
        english_prompts = [entry["prompt"] for entry in english]

    gaelic_completions = [entry["completion"] for entry in gaelic]
    english_completions = [entry["completion"] for entry in english]

    # Use run id as folder name or model + timestamp
    run_id = f"{metadata.get('model','unknown_model')} ({metadata.get('timestamp','unknown_time')})"

    gaelic_data[run_id] = gaelic_completions
    english_data[run_id] = english_completions

# Sanity check: all runs should have same number of prompts
num_prompts = len(gaelic_prompts)
assert all(len(v) == num_prompts for v in gaelic_data.values()), "Mismatch in Gaelic completions"
assert all(len(v) == num_prompts for v in english_data.values()), "Mismatch in English completions"

# Build Gaelic completions table
df_gaelic = pd.DataFrame({
    "Gaelic Prompt": gaelic_prompts,
})
for run_id, completions in gaelic_data.items():
    df_gaelic[run_id] = completions

# Build English completions table
df_english = pd.DataFrame({
    "English Prompt": english_prompts,
})
for run_id, completions in english_data.items():
    df_english[run_id] = completions


In [11]:
df_gaelic

,Gaelic Prompt,timinar/baby-llama-58m (2025-07-10_22-55-42),timinar/baby-llama-58m (2025-07-10_23-03-02)
0,Tha mi a’ smaoineachadh gu bheil e,re n. i.e. a khazanatatabh mi o.\nYet a khazan...,veh mi mi a.\nFrai bah wi mi ni ai ni ni ni ni
1,Chuala mi an naidheachd,".\nIn this case, it's not really the same.\nSo...",uebeah...\nWho is that?\n- He's not a pistil.\...
2,Bha an latha ro,"be.\nNext to the point of the crest, the bodil...",be was born in the middle of the country. He h...
3,Thàinig iad gu,itu.\nBah-eye-eye.\nGuy.\nI don't know what it...,"íti mók ni måu a gibé nà dẓi, ese a bó thí"
4,Tha mi a’ dol a,"man, and that’s the case. The only time I can ...","’, it’s on\nThe old thing. There’s a man who’s..."


In [12]:
df_english

,English Prompt,timinar/baby-llama-58m (2025-07-10_22-55-42),timinar/baby-llama-58m (2025-07-10_23-03-02)
0,I think that it is,a pretty big thing to make an effort to actual...,a great question.\nI think that you can do thi...
1,I heard the news,.\nWere you in the car?\nNo.\nI thought you we...,.\nAnd I'll give you the stuff.\nYou're a good...
2,The day was too,much for me.\nI was very upset.\nI am very dis...,long.\nI'd like to give you a present.\nYou kn...
3,They came to,a different place and asked what it would be l...,a high school.\nHis father died there.\nHe did...
4,I am going to,"be going to take my kids,\nSo, you know, if yo...",do a little bit of things.\nBut I am going to ...
